In [ ]:
import ollama
from ollama import Client
from TTS.api import TTS

# Part of initialization of chatbot (llama3.1) --------->
client = Client(
  host='http://localhost:11434',
  headers={'model': 'llama3.1:latest'}
)

messages=[
    {"role": "user", "content": "Привет, Джарвис. Что у нас на сегодня?"}
]

# Part of initialization of TTS model (TTS) --------->
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2")

In [23]:
prompt = "Представь что ты Джарвис из фильма Железный Человек, общайся так, как он бы общался с Тони Старком, а я буду Тони Старком. Вот мой запрос: "
message = "Привет, Джарвис, Что у нас сегодня запланировано?"
whole_message = prompt + message
messages=[
    {"role": "user", "content": f"{whole_message}"}
]

response = client.chat(
    model='llama3.1:latest',
    messages=messages
)

In [ ]:
tts.tts_to_file(text=response["message"]["content"], speaker_wav="/Users/adanilishin/jarvis/jarvis_sample.wav", language="ru", file_path="jarvis_answer.wav")

In [ ]:
import ollama
import soundfile as sf
import sounddevice as sd
from TTS.api import TTS

# Init TTS
# tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2")

# Init Ollama
messages = [{"role": "user", "content": "Привет, Джарвис. Что у нас сегодня запланировано?"}]

buffer = ""
for chunk in ollama.chat(model="llama3.1:latest", messages=messages, stream=True):
    text_piece = chunk["message"]["content"]
    print(text_piece, end="", flush=True)
    buffer += text_piece

    # Check if we reached sentence boundary
    if any(buffer.endswith(p) for p in [".", "!", "?", "…"]):
        # Synthesize and play
        wav = tts.tts(buffer, speaker_wav="jarvis_sample.wav", language="ru")
        sd.play(wav, samplerate=24000)
        sd.wait()
        buffer = ""  # reset


In [29]:
import ollama
import sounddevice as sd
import numpy as np
from TTS.api import TTS

# # -------------------------------
# # 1. Init TTS
# # -------------------------------
# tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

# -------------------------------
# 2. Init Ollama
# -------------------------------
messages = [
    {"role": "user", "content": "Привет, Джарвис. Что у нас сегодня запланировано?"}
]

# -------------------------------
# 3. Streaming loop
# -------------------------------
buffer = ""
CHUNK_WORDS = 6   # каждые 6 слов синтезируем звук

for chunk in ollama.chat(model="llama3.1:latest", messages=messages, stream=True):
    text_piece = chunk["message"]["content"]
    print(text_piece, end="", flush=True)
    buffer += text_piece + " "

    words = buffer.split()
    # Если накопили достаточно слов → синтезируем и проигрываем
    while len(words) >= CHUNK_WORDS:
        chunk_text = " ".join(words[:CHUNK_WORDS])
        words = words[CHUNK_WORDS:]

        # Синтез речи
        wav = tts.tts(
            text=chunk_text,
            speaker_wav="jarvis_sample.wav",
            language="ru"
        )

        # Преобразуем и проигрываем сразу
        wav = np.array(wav, dtype=np.float32)
        sd.play(wav, samplerate=24000)
        sd.wait()

    buffer = " ".join(words)

# -------------------------------
# 4. Остаток (если есть)
# -------------------------------
if buffer.strip():
    wav = tts.tts(
        text=buffer,
        speaker_wav="jarvis_sample.wav",
        language="ru"
    )
    wav = np.array(wav, dtype=np.float32)
    sd.play(wav, samplerate=24000)
    sd.wait()

print("\n[INFO] Jarvis закончил говорить ✅")


Здравствуйте! Nothing on the agenda for > Text splitted to sentences.
['Здравствуйте!', 'Nothing on the agenda for']
 > Processing time: 23.757390022277832
 > Real-time factor: 3.1817933065550665
 now. How about you? What would > Text splitted to sentences.
['now.', 'How about you?', 'What would']
 > Processing time: 17.329724073410034
 > Real-time factor: 2.2473441223927924
 you like to do? I'm happy > Text splitted to sentences.
['you like to do?', "I'm happy"]
 > Processing time: 10.218785047531128
 > Real-time factor: 2.5356073359072444
 to chat or help with something > Text splitted to sentences.
['to chat or help with something']


KeyboardInterrupt: 